In [2]:
import spikeinterface.full as si
#import spikeinterface.core as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spiketoolkit
import spikeinterface.widgets as sw
import numpy as np
from convert_positions_in_tones import *
from utils_extraction import *
import numpy as np
import spikeinterface
import zarr as zr
import os
from  pathlib import Path
import tqdm
import spikeinterface.full as si
from spikeinterface.sortingcomponents.peak_detection import detect_peaks
from probeinterface import Probe, ProbeGroup
import matplotlib
import pickle
matplotlib.use('Agg')


In [3]:
sr=30e3
# adresse pour OSCYPEK : /mnt/working2/felicie/data2/eTheremin/OSCYPEK/OSCYPEK

root = '/mnt/working2/felicie/data2/eTheremin/ALTAI/ALTAI_20240712_SESSION_00/'
path = root+'headstage_0/' 
#path=root
#neural_data = np.load(root +'/neural_data.npy')
#pour Burrata : s
neural_data = np.load(path +'/neural_data.npy')
sig = neural_data

In [6]:
def get_fma_probe():
    """ Implements the FMA probe using Probeinterface.
        Particular care has to be taken on how the headstage is connected on the probe.
    """
    ### The following distances are in mm:
    inter_hole_spacing = 0.4  # along one row
    inter_row_spacing = np.sqrt(0.4**2-0.2**2)  # between row/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_01/headstage_1/psth_figure_spikeinterface.png

    # We need to remove the ground from these positions:
    mask = np.zeros((16, 2), dtype=bool)
    mask[[0, 15], [0, 1]] = True
    positions = positions[np.logical_not(mask.reshape(-1))]

    probe = Probe(ndim=2, si_units='um')
    probe.set_contacts(positions=positions,
                       shapes='circle', shape_params={'radius': 100})
    polygon = [(0, 0), (0, 16000), (800, 16000), (800, 0)]
    probe.set_planar_contour(polygon)

    ## This mapping is ordered from 32 to 1, so we use[::-1] to invert it and make it 1 to 32
    mapping = np.arange(32).reshape(16, 2)[::-1].reshape(-1)
    mapping = mapping[np.logical_not(mask.reshape(-1))]

    probe.set_device_channel_indices(mapping)

    return probe

In [12]:
c0 = sig[0]
c1 = sig[1]
c01 = sig[0:2]

In [25]:
n_cpus = os.cpu_count()

full_raw_rec = se.NumpyRecording(traces_list=np.transpose(c01), sampling_frequency=sr)
#raw_rec = full_raw_rec.set_probe(probe)
#raw_rec = raw_rec.remove_channels(["CH2"])
recording_cmr = si.common_reference(full_raw_rec, reference='global', operator='median')
recording_f = si.bandpass_filter(full_raw_rec, freq_min=300, freq_max=9000)

In [26]:
recording_f

BandpassFilterRecording: 2 channels - 30.0kHz - 1 segments - 100,971,264 samples 
                         3,365.71s (56.10 minutes) - int16 dtype - 385.17 MiB

In [27]:
traces = recording_f.get_traces()

# Sauvegarder les traces dans un fichier .npy
np.save(path+'recording_filtered.npy', traces)

In [4]:
path = '/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_04/headstage_0/'
ANALOG_TRIGGERS_MAPPING = {"MAIN": 1, "PLAYBACK": 0}

In [40]:
def create_tt_tracking_only(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False):
    """"
    Fonction ok dans le cas où on a pas 
def read_log_file(folder)::
        tt_pb = [tones_tr, an_tr_trigs]
    else:
        min_length = min(len(tones_tr), len(an_tr_trigs))
        tt_tr = [tones_tr[:min_length], an_tr_trigs[:min_length]]
    # Create the final object
    tt_object = {
        'n_iter': n_iter,
        'recording_length': recording_length,
        'tr_0' : tt_tr  # Add the combined triggers and tones dictionaries
    }
    np.savez(path+'/tt.npz', tt_object)
    return tt_object

In [6]:
create_tt_playback_only(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False)

/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_04/headstage_0/trig_dig_chan_{}.npy
[False False False ... False False False]


{'n_iter': 1817,
 'recording_length': 94549,
 'pb_0': [array([2000., 2317., 2684., ..., 1111., 1287., 1111.]),
  array([ 1021288,  1075583,  1228045, ..., 17833658, 17834408, 17834618])]}

In [7]:
tt = np.load(path+'/tt.npz', allow_pickle=True)

In [11]:
tt['arr_0']

array({'n_iter': 919, 'recording_length': 0, 'pb_0': [array([ 4172.,  8701.,  3109.,   397., 21024.,  1287.,   533.,   255.,
        3601.,   220.,  5598.,   190.,  2317.,   617.,   460.,  4832.,
        1111., 18149.,  7511.,  1490.,  6484., 13525.,   715., 11676.,
       15667.,  2000.,   959., 10079.,   828.,  2684.,   296.,  1727.,
         343., 11676.,  7511., 10079., 13525.,  1490.,   343.,  3109.,
        4832.,   533.,   460.,   959., 21024.,  4172.,   828.,  2684.,
        6484., 18149.,  8701.,  2317.,   296.,  3601., 15667.,   255.,
        1727.,  1287.,  1111.,   617.,  2000.,   397.,  5598.,   190.,
         220.,   715.,  3109.,   533.,  7511., 15667.,  3601., 21024.,
        2000.,   959.,  6484.,   617.,   828.,  1727.,  2317.,   220.,
         343.,   397.,  1287., 10079., 13525., 11676.,   715.,   255.,
        4832.,  2684.,  4172.,   296.,  5598., 18149.,  1111.,  1490.,
         460.,   190.,  8701.,   828.,   397.,  6484.,  1727.,   255.,
        3601., 10079., 

In [6]:
def create_tt_playback_only(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False):
    """"
    Fonction ok dans le cas où on a pas de mock triggers 
    
    """
    
    tones_pb = get_tones_for_tt_pbOnly(path, dowehavemock)
    n_iter = len(tones_pb)
    
    an_tr_trigs, an_pb_trigs, an_mck_trigs, digital_channels = get_triggers_for_tt(path, ANALOG_TRIGGERS_MAPPING, dowehavemock)
    recording_length = extract_recording_length(digital_channels)
    if len(tones_pb) == len(an_pb_trigs):
        tt_pb = [tones_pb, an_pb_trigs]
    else:
        min_length = min(len(tones_pb), len(an_pb_trigs))
        tt_pb = [tones_pb[:min_length], an_pb_trigs[:min_length]]
    # Create the final object
    tt_object = {
        'n_iter': n_iter,
        'recording_length': recording_length,
        'pb_0' : tt_pb  # Add the combined triggers and tones dictionaries
    }
    np.savez(path+'/tt.npz', tt_object)
    return tt_object

In [21]:
an_tr_trigs, an_pb_trigs, an_mck_trigs, digital_channels = get_triggers_for_tt(path, ANALOG_TRIGGERS_MAPPING)

Error:  Glob in folder should be of length 1.


UnboundLocalError: local variable 'd' referenced before assignment

In [5]:
print(an_tr_trigs, an_pb_trigs, an_mck_trigs, digital_channels)

[] [  420025   443385   465945   488665   511385   533785   556505   578905
   601464   624344   646744   669304   691864   714264   736824   759383
   781783   804343   826743   849303   871863   894263   916823   939382
   961942   984342  1006742  1029302  1051862  1074262  1096822  1119381
  1141781  1164341  1186901  1209301  1231861  1254261  1276821  1299380
  1321780  1344340  1366900  1389300  1411860  1434260  1456820  1479379
  1501779  1524339  1546899  1569299  1591859  1614259  1636819  1659378
  1681938  1704338  1727058  1749938  1772658  1795378  1817938  1840337
  1862897  1886097  1908817  1931857  1954737  1977617  2000016  2022736
  2045616  2068176  2090576  2113136  2135536  2158096  2180655  2203215
  2225615  2248175  2270575  2293135  2315535  2338095  2360654  2383054
  2405614  2428174  2450574  2473134  2495534  2518094  2540653  2563053
  2585613  2608173  2630573  2653133  2675853  2698573  2721132  2744012
  2766572  2788972  2811532  2833932  2856652  2

In [18]:
print(an_pb_trigs)

[ 1021288  1075583  1228045 ... 17833658 17834408 17834618]


In [19]:
tt=create_tt_playback_only(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False)

/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_04/trig_dig_chan_{}.npy
[False False False ... False False False]


In [20]:
an_pb_trigs

{'n_iter': 1817,
 'recording_length': 94549,
 'pb_0': [array([2000., 2317., 2684., ..., 1111., 1287., 1111.]),
  array([ 1021288,  1075583,  1228045, ..., 17833658, 17834408, 17834618])]}

In [32]:
print(tt_pb['arr_0'])

TypeError: list indices must be integers or slices, not str

In [15]:
def associate_tones_and_triggers_pbOnly(tones, triggers):
   
    # Create an empty list to store associations
    associations = []

    # Iterate over each subarray of tones
        # Get the length of the current subarray
    subarray_length = len(tones)
        
        # Take the corresponding triggers for the current subarray
    subarray_triggers = triggers[:subarray_length]
        
        # Associate triggers with tones in a dictionary
    subarray_associations = dict(zip(tones, subarray_triggers))
        
        # Add associations for the current subarray to the list
    associations.append(subarray_associations)
        
        # Remove the triggers used for this subarray
    triggers = triggers[subarray_length:]
    return associations

In [19]:
tt_pb = associate_tones_and_triggers_pbOnly(tones_pb, an_pb_trigs)

In [13]:
len(an_pb_trigs)

2436

In [12]:
an_tr_trigs, an_pb_trigs, an_mck_trigs, digital_channels = get_triggers_for_tt(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False)

/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240420_SESSION_01/trig_dig_chan_{}.npy
[False False False ...  True  True  True]


In [14]:
tt_object, tt_pb=create_tt_playback_only(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False)

/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240420_SESSION_01/trig_dig_chan_{}.npy
[False False False ...  True  True  True]


In [15]:
tt

{'n_iter': 2439,
 'recording_length': 133286,
 'pb_0': [{2000.0: 24444055,
   2317.0: 24443695,
   2684.0: 24342085,
   1727.0: 24559197,
   1490.0: 25116268,
   1287.0: 25138378,
   1111.0: 24883739,
   959.0: 24619737,
   3109.0: 20954475,
   3601.0: 20946015,
   4172.0: 20936805,
   4832.0: 20926155,
   5598.0: 20915265,
   6484.0: 20904765,
   7511.0: 20895195,
   8701.0: 20885450,
   10079.0: 20867415,
   828.0: 22365229,
   715.0: 22284949,
   617.0: 22219880,
   533.0: 21276767,
   11676.0: 20836335,
   13525.0: 20803708}]}

In [ ]:
create_tt_playback_only(path, ANALOG_TRIGGERS_MAPPING, dowehavemock=False)

/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_04/trig_dig_chan_{}.npy
[False False False ... False False False]


TypeError: object of type 'numpy.int32' has no len()

In [ ]:
tt = np.load(path+'/tt.npz', allow_pickle=True)

In [ ]:
tt['arr_0']

In [ ]:

def find_json_file(directory_path):
    # Use glob to find all JSON files in the directory
    json_files = glob.glob(os.path.join(directory_path, "*.json"))
    
    # Check if any JSON files were found
    if json_files:
        return json_files[0]
    else:
        return None

In [ ]:
def get_block_numbers(json_file):
    """"
    Extraire le nombre de blocs dans une session à partir du json
    arg : le json file
    output : int(numero du premier bloc),  int(numéro du dernier bloc)
    """
    block_keys = [key for key in json_file if key.startswith("Block_00")]
    block_numbers = [int(block.split('_')[1]) for block in block_keys]
    first_block_number = min(block_numbers)
    max_block_number = max(block_numbers)
    return first_block_number, max_block_number

In [ ]:
def check_block_ended_correctly(data):
    experiment_results = {}
    for block_key, block_data in data.items():
        if block_key.startswith("Block_"):
            experiment_results[block_key] = block_data.get("Experiment ended correctly", None)
    return experiment_results

In [ ]:
def iterate_log_for_tones_fn(folder, log, allowed_kw, key_to_fetch):
    """
    Itère dans le .json à la recherche des noms de fichiers sons.
    :param key_to_fetch:
    :param folder:
    :param log:
    :param allowed_kw:
    :return:
    """
    tones_folder = os.path.join(folder, log["Tones folder"])
    tones_fn = {kw: list() for kw in allowed_kw}
    print(tones_folder)
    sub_log = log[key_to_fetch]
    for kw in allowed_kw:
        for key in sub_log.keys():
            if re.match(kw, key):
                tones_fn[kw].append(os.path.join(tones_folder, sub_log[key]["Tones_fn"]))
                if kw == "playback":
                    tones_fn["mock"].append(os.path.join(tones_folder, sub_log[key]["Mock_fn"]))
    return tones_fn

In [ ]:
def get_tones(folder, log, allowed_kw, key_to_fetch):
    """
    Charge les fichiers dans un np.ndarray.
    :param folder:
    :param log:
    :param allowed_kw:
    :param key_to_fetch:
    :return:
    """
    tones_fn = iterate_log_for_tones_fn(folder, log, allowed_kw, key_to_fetch)
    tones_values = {kw: list() for kw in allowed_kw}
    # C'est à dire : fichier dans le log, mais absent du dossier car vide.
    # Là, je cherche à charger les .bin dans un dictionnaire de listes.
    for key in tones_fn.keys():
        tones_file_for_type = tones_fn[key]
        for file in tones_file_for_type:
            if not os.path.exists(file):
                tones = np.empty(0)
            else:
                tones = np.fromfile(file, dtype=np.double)
            tones_values[key].append(tones)
    return tones_values

In [ ]:
#analog_in et digital_in
#ANALOG_TRIGGERS_MAPPING = {"MAIN": 1, "PLAYBACK": 0, "MOCK": 3, "TARGET": 2}
#ANALOG_TRIGGERS_MAPPING = {"MAIN": 0, "PLAYBACK": 1, "MOCK": 2}
ANALOG_TRIGGERS_MAPPING = {"MAIN": 0, "PLAYBACK": 1}

#hje checke que je suis en v2 dans le json
json_file_path = find_json_file(path)

with open(json_file_path, 'r') as f:
        json_file = json.load(f)

blocks_ended_correctly = list(check_block_ended_correctly(json_file))

dig_trig_mapping = get_digital_mapping(json_file)


# for le nombre de blocks
first_block, last_block = get_block_numbers(json_file)

# pour un block :
n_blocks = last_block-first_block+1

#n_blocks =5
#extraire les canaux analogiques et digitaux avec le bon mapping des canaux digitaux
#triggers = extract(path)

#extraire positions et tons block par block
positions_tr, tones_tr, positions_pb, tones_pb, tones_mck = [], [], [], [], []
for block in range(0, n_blocks):
    print(block)
    try:
        if blocks_ended_correctly[block]:
            print('block ended correctly')
            # extraire les positions et les tons
            tones_tr.append(get_tones(path, json_file, ['tracking', 'playback', 'mock'], f"Block_00{block}")['tracking'][0])
            #positions_tr.append(get_positions(path, json_file, ['tracking', 'playback'], f"Block_00{block}")['tracking'][0])
            #tones = extract_tones_path(json_file, block, condition)


        #en playback : convertir les positions en tones pour avoir les MOCK
        # mock = get_tones_from_position (nom donné au pif)
            tones_pb.append(get_tones(path, json_file, ['tracking', 'playback', 'mock'], f"Block_00{block}")['playback'][0])
            positions_pb.append(get_positions(path, json_file, ['tracking', 'playback'], f"Block_00{block}")['playback'][0])
            tones_mck.append(get_tones(path, json_file, ['tracking', 'playback', 'mock'], f"Block_00{block}")['mock'][0])
        else:
            break
    except:
        pass

0
block ended correctly
/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_04/tones
1
block ended correctly
/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240419_SESSION_04/tones


In [ ]:
tones_pb

[]

In [ ]:
# découper les triggers en bloc

#extraire les canaux analogiques et digitaux avec le bon mapping des canaux digitaux
triggers = extract(path, ANALOG_TRIGGERS_MAPPING)
analog_channels = triggers['ANALOG']
digital_channels = triggers['DIGITAL']

tr_trigs = analog_channels['MAIN']
pb_trigs = analog_channels['PLAYBACK']
#mck_trigs = analog_channels['MOCK']

In [ ]:
tr_trigs

In [ ]:
def save_recording_length(folder, length):
    with open(os.path.join(folder, "recording_length.bin"), "w") as f:
        f.write('{:03d}\n'.format(length))

In [ ]:
def extract_recording_lenght(triggers):
    return len(triggers['BASLER'])
    

In [ ]:
def associate_tones_and_triggersv2(tones, triggers, condition):
    """""
    Dans le cas où on a les mocks triggers
    
    """
    # Create an empty dictionary to store associations
    tt = {}
    if condition == 'tracking':
        cond = 'tr'
    elif condition == 'playback':
        cond = 'pb'
    elif condition == 'mock':
        cond = 'mck'
    # Iterate over each index and subarray of tones
    for i, block_tones in enumerate(tones): 
        # Get the length of the current subarray
        subarray_length = len(block_tones)
        
        # Take the corresponding triggers for the current subarray
        subarray_triggers = triggers[:subarray_length]
        
        # Associate triggers with tones in a dictionary
        tt[f"{cond}_{i}"] = [block_tones, subarray_triggers]
        
        # Remove the triggers used for this subarray
        triggers = triggers[subarray_length:]
    return tt


In [ ]:
def create_tt(path, analog_channels, digital_channels, tones_tr, tones_pb):
    """"
    Fonction ok dans le cas où on a pas de mock triggers 
    
    """
    n_iter = len(tones_pb)
    tr_trigs = analog_channels['MAIN']
    pb_trigs = analog_channels['PLAYBACK']
    
    # Assuming you have functions to create the arrays 'order', 'n_iter', 'recording_length', 'keys', and 'numbers'
    # Assuming you have a function to extract recording length from the path
    recording_length = extract_recording_lenght(digital_channels)
    
    # Create the arrays for 'pb_' and 'tr_' using the provided function
    tt_tr = associate_tones_and_triggersv2(tones_tr, tr_trigs, 'tracking')
    tt_pb = associate_tones_and_triggersv2(tones_pb, pb_trigs, 'playback')
    
    # Combine 'tt_tr' and 'tt_pb' dictionaries
    tt_tr.update(tt_pb)
    
    # Create the final object
    tt_object = {
        'n_iter': n_iter,
        'recording_length': recording_length,
        **tt_tr  # Add the combined triggers and tones dictionaries
    }
    
    return tt_object


In [ ]:
def create_tt(path, analog_channels, digital_channels, tones_tr, tones_pb, tones_mck):
    """"
    Dans le cas où on a les mock triggers 
    
    """
    n_iter = len(tones_pb)
    tr_trigs = analog_channels['MAIN']
    pb_trigs = analog_channels['PLAYBACK']
    mck_trigs = analog_channels['MOCK']
    
    # Assuming you have functions to create the arrays 'order', 'n_iter', 'recording_length', 'keys', and 'numbers'
    # Assuming you have a function to extract recording length from the path
    recording_length = extract_recording_lenght(digital_channels)
    
    # Create the arrays for 'pb_' and 'tr_' using the provided function
    tt_tr = associate_tones_and_triggersv2(tones_tr, tr_trigs, 'tracking')
    tt_pb = associate_tones_and_triggersv2(tones_pb, pb_trigs, 'playback')
    tt_mck = associate_tones_and_triggersv2(tones_mck, mck_trigs, 'mock')
    
    # Combine 'tt_tr' and 'tt_pb' dictionaries
    tt_tr.update(tt_pb)
    tt_tr.update(tt_mck)
    
    # Create the final object
    tt_object = {
        'n_iter': n_iter,
        'recording_length': recording_length,
        **tt_tr  # Add the combined triggers and tones dictionaries
    }
    
    return tt_object

In [ ]:
tt = create_tt(path, analog_channels, digital_channels, tones_tr, tones_pb)


In [ ]:
for i in range(len(tt)):
    print('tr_ ', len(tt[f'tr_{i}'][0]))
    print('pb_ ',len(tt[f'pb_{i}'][0]))

In [ ]:
print(np.mean(np.diff(tt['pb_1'][1:]) -np.diff(tt['tr_1'][1])))


In [ ]:
Experiment/BURRATA/BURRATA_20240501_SESSION_00/accelerometer.npy Experiment/BURRATA/BURRATA_20240501_SESSION_00/analog_in.npy Experiment/BURRATA/BURRATA_20240501_SESSION_00/dig_in.npy# je modifie le tt à la main pour que ca colle 
tt['pb_1'][0] = tt['pb_2'][0][1:]

In [ ]:
np.savez(path+'tt.npz', tt)
print('all good')

In [ ]:
tt['tr_6']

In [ ]:
from get_data import *

In [ ]:
def extract_tt(folder):
    tt_file = os.path.join(folder, "tt.npz")
    assert(os.path.exists(tt_file)), "No tt file."
    seq = SequenceTT(folder=folder)
    return seq

In [ ]:
extract_data(path, analog_channels=None, digital_channels=None, compatibility=True)

In [ ]:
DIGITAL_TRIGGERS_MAPPING = {"Basler": 0, "Sounds": 1}
ANALOG_TRIGGERS_MAPPING = {"MAIN": 0, "PLAYBACK": 1, "MOCK": 2}


def check_plot_folder_exists(directory):
    path = os.path.join(directory, "plot")
    if not os.path.exists(path):
        os.mkdir(path)
    return path

analog_channels = triggers['ANALOG']
digital_channels = triggers['DIGITAL']

tr_trigs = analog_channels['MAIN']
pb_trigs = analog_channels['PLAYBACK']
mck_trigs = analog_channels['MOCK']

def extract_data(folder, triggers, compatibility=False):
    seq, length = check_already_extracted(folder)
    seq, length = False, False
    if seq and length:
        return extract_tt(folder)
    else:
        dig_file = os.path.join(folder, "dig_in.npy")
        analog_file = os.path.join(folder, "analog_in.npy")

        # todo: faire fonction pour le calcul de la longueur de l'enregistrement.
        if length:
            length = get_recording_length(folder)
        else:
            assert (os.path.exists(dig_file)), "No digital triggers file in directory."
            d_trigs = np.load(os.path.join(folder, "dig_in.npy"))
            length = d_trigs.shape[1]
            save_recording_length(folder, length)

        out_analog = triggers['ANALOG']
        out_digital = triggers['DIGITAL']
        # FIN de la fonction à créer.
        if seq:
            seq = extract_tt(folder)
        else:
            triggers = dict()
            tr_trigs = analog_channels['MAIN']
            pb_trigs = analog_channels['PLAYBACK']
            mck_trigs = analog_channels['MOCK'] 
   
            if analog_channels is None:
                triggers["tracking"] = out_analog["tracking"]
                if not compatibility:
                    triggers["mock"] = out_analog["mock"]
                triggers["playback"] = out_analog["playback"]
            if compatibility:
                seq = get_data_6(triggers, folder)
            else:
                seq = get_data_5(triggers, folder)
        seq.set_recording_length(length)
        seq.save(folder)
        return seq

In [ ]:
def get_data_6(triggers, folder):
    d_out = dict()
    l_tracking, l_mock, l_pb, l_warmup = fetch_tones(folder)
    assert (len(l_pb) == len(l_tracking) == len(l_mock))
    n_iter = len(l_pb)
    fn = os.path.join(folder, "durations.json")
    if os.path.exists(fn):
        with open(fn, "r") as f:
            d = json.load(f)
        duration_tr = d["tracking"]
    else:
        duration_tr = 5
        duration_warmup = 5
        duration_warmdown = 5

    c = 0
    sequence = Sequence()
    #sequence.add(XPSingleton("warmup", c, 0, duration_warmup, tones=l_warmup[0]))
    for i in range(n_iter):
        sequence.add(XPSingleton("tracking", c, i, duration_tr, tones=l_tracking[i]))
        c += 1
        sequence.add(XPSingleton("mock", c, i, duration_tr, tones=l_mock[i]))
        c += 1
        sequence.add(XPSingleton("playback", c, i, duration_tr, tones=l_pb[i]))
        c += 1

    d_out = divide_triggers_2(triggers, sequence, d_out, n_iter=n_iter)

In [ ]:
def triggers_tones_inspection_2(tt_seq, triggers, sequence, n_iter):
    pb_triggers = triggers["playback"]
    tr_triggers = triggers["tracking"]
    mck_triggers = triggers["mock"]
    digital_triggers = triggers["dig"]
    pb_duration = sequence.get_duration_for("playback")

    tr_done = False
    pb_done = False
    mck_done = False

    s_pb = sequence.get_n_tones_for("playback")

    s_mck = sequence.get_n_tones_for("mock")

    s_tr = sum([sequence.get_n_tones_for(elt) for elt in ["tracking", "warmdown", "warmup"]])

    if s_tr == len(tr_triggers):
        tt_seq = if_complete_2(tr_triggers, sequence, tt_seq, type_of=None)
        tr_done = True

    if s_pb == len(pb_triggers):
        tt_seq = if_complete_2(pb_triggers, sequence, tt_seq, type_of="playback")
        pb_done = True

    if s_mck == len(mck_triggers):
        tt_seq = if_complete_2(mck_triggers, sequence, tt_seq, type_of="mock")
        mck_done = True

    where_to_withdraw = list()
    where_to_append = dict()
    if tr_done and pb_done and mck_done:
        return tt_seq, tr_done, pb_done, mck_done

    if not pb_done:
        # assembler les .bin tones
        # regarder quels triggers ne sont pas représentés dans les triggers analogiques et dans les triggers digitaux.

        p = deepcopy(pb_triggers)
        d = deepcopy(digital_triggers)
        d_corr_0 = synchronize_step(d, tr_triggers)
        d = d[np.isnan(d_corr_0[1])]
        a = list()  # triggers analogiques
        b = list()  # triggers digitaux
        for i in range(n_iter):
            start = p[0]
            end = p[0] + sequence.get_duration_for("playback")
            idx_p = np.less_equal(p, end)
            idx_d = np.logical_and(d >= start - 10000, d <= end)

            b.append(d[idx_d])
            a.append(p[idx_p])
            p = p[~idx_p]
        # 1) Assembler
        for i in range(n_iter):
            xp = sequence.get_xp_number("playback", i)
            _tones = xp.tones
            # _tones = np.vstack((np.full_like(_tones, i), _tones))
            # concat_pb_tones.append(_tones)

            if len(_tones) != len(a[i]):
                d_corr = synchronize_step(b[i], a[i], begin=False)
                if has_nan(d_corr[1]):
                    _tones = _tones[~np.isnan(d_corr[1])]
                else:
                    tr = sequence.get_xp_number("tracking", i).tones
                    if len(tr) == len(a[i]):
                        _tones = tr

            tt_seq.add(Pair(_tones, a[i], xp.type, number=xp.n, order=xp.order))

        pb_done = True

    if not tr_done:
        cp_tr_t = deepcopy(tr_triggers)
        # attraper les premiers triggers pour le warmup.
        xp = sequence.get_xp_number("warmup", 0)
        t0 = tr_triggers[0]
        idx = np.equal(xp.tones, 0)

        # 1) on sors le warmup et le warmdown.
        if sum(idx) > 0:
            xp.tones = xp.tones[~idx]
        wp_duration = sequence.get_duration_for("warmup")
        idx = np.less_equal(cp_tr_t, pb_triggers[0] - pb_duration)
        warmup_triggers = cp_tr_t[idx]
        if len(warmup_triggers) != len(xp.tones):
            if len(warmup_triggers) > len(xp.tones):
                d = len(warmup_triggers) - len(xp.tones)
                warmup_triggers = warmup_triggers[d:]
            else:
                # d = len(xp.tones) - len(warmup_triggers)
                xp.tones = xp.tones[:-1]

        tt_seq.add(Pair(xp.tones, warmup_triggers, xp.type, number=xp.n, order=xp.order))
        cp_tr_t = cp_tr_t[~idx]
        idx = np.greater_equal(cp_tr_t, pb_triggers[-1])
        xp = sequence.get_xp_number("warmdown", 0)
        wd_triggers = cp_tr_t[idx]
        tt_seq.add(Pair(xp.tones, wd_triggers, xp.type, number=xp.n, order=xp.order))
        cp_tr_t = cp_tr_t[~idx]

        # 2) on fait l'extraction des blocks tracking.
        if pb_done:
            l_tr_blocks = list()
            for i in range(n_iter):
                tt_after = tt_seq.get_xp_number("playback", i).triggers[0]
                if i > 0:
                    tt_before = tt_seq.get_xp_number("playback", i - 1).triggers[-1]
                    idx = np.logical_and(tt_before <= cp_tr_t, tt_after >= cp_tr_t)
                elif i == 0:
                    idx = np.logical_and(t0 + wp_duration <= cp_tr_t, tt_after >= cp_tr_t)
                l_tr_blocks.append(cp_tr_t[idx])
                cp_tr_t = cp_tr_t[~idx]

            # retirer indice 0 dans les triggers du tracking, si nécessaire.
            for i, elt in enumerate(l_tr_blocks):
                xp = sequence.get_xp_number("tracking", i)
                print(len(xp.tones), len(elt))
                if i == 0 and len(xp.tones) > len(elt):  # cas rare où le ton du warmup se trouve dans le tracking.
                    xp.tones = xp.tones[1:]

                if len(xp.tones) + 1 == len(elt):  # cas commun où le ton du mock se trouve dans le tracking.
                    if i != 0:
                        where_to_withdraw.append(i - 1)
                        mck_xp = sequence.get_xp_number("mock", i - 1)
                        xp.tones = np.hstack((mck_xp.tones[-1], xp.tones))

                if len(elt) + 1 == len(xp.tones):
                    if i != 0:
                        where_to_append[i] = xp.tones[-1]
                        xp.tones = xp.tones[:-1]

                tt_seq.add(Pair(xp.tones, l_tr_blocks[i], xp.type, number=xp.n, order=xp.order))

            tr_done = True

    if not mck_done:
        mck_duration = sequence.get_duration_for("mock")
        l_mck_blocks = list()
        cp_mck_t = deepcopy(mck_triggers)
        if tr_done:
            for i in range(n_iter):
                if len(sequence.get_xp_number("mock", i).tones) == 0:
                    l_mck_blocks.append([])
                    continue
                if i == n_iter - 1:
                    start = tt_seq.get_xp_number("tracking", i).triggers[-1]
                    stop = tt_seq.get_xp_number("warmdown", 0).triggers[0]
                else:
                    start = tt_seq.get_xp_number("tracking", i).triggers[-1]
                    stop = tt_seq.get_xp_number("tracking", i + 1).triggers[0]
                idx = np.logical_and(start <= cp_mck_t, stop >= cp_mck_t)
                l_mck_blocks.append(cp_mck_t[idx])
                cp_mck_t = cp_mck_t[~idx]
        else:
            for i in range(n_iter):
                end = cp_mck_t[0] + mck_duration
                if len(sequence.get_xp_number("mock", i).tones) == 0:
                    l_mck_blocks.append([])
                    continue
                idx = np.less_equal(cp_mck_t, end)
                l_mck_blocks.append(cp_mck_t[idx])
                cp_mck_t = cp_mck_t[~idx]

        for i, elt in enumerate(l_mck_blocks):

            xp = sequence.get_xp_number("mock", i)

            if i in where_to_withdraw:
                tt_seq.add(Pair(xp.tones[:-1], l_mck_blocks[i], xp.type, number=xp.n, order=xp.order))

            elif i in where_to_append.keys():
                tt_seq.add(Pair(xp.tones, l_mck_blocks[i], xp.type, xp.n, xp.order))

            else:
                if len(xp.tones) == len(l_mck_blocks[i]):
                    tt_seq.add(Pair(xp.tones, l_mck_blocks[i], xp.type, number=xp.n, order=xp.order))
                else:
                    tt_seq.add(Pair(xp.tones[:-1], l_mck_blocks[i], xp.type, number=xp.n, order=xp.order))

    return tt_seq

voila c'est tout depuis le debut pour faire le fichier tt quand il n'y a pas de mock triggers !

In [ ]:
import numpy as np
file_path = '/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240503_SESSION_02/headstage_1/tt.npz'
#file_path = '/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/FRINAULT/FRINAULT_20230223/tt.npz'
def read_npz_file(file_path):
    format_tt = []
    # Load the .npz file
    npz_file = np.load(file_path, allow_pickle=True)
    
    # Access the arrays stored in the .npz file
    for array_name in npz_file:
        array_data = npz_file[array_name]
        format_tt.append(array_data)
    return format_tt
        

# Example usage:

mytt = read_npz_file(file_path)
